In [ ]:
import pandas as pd

df = pd.read_csv("C:\\Users\\ncvn\\.cache\\kagglehub\\datasets\\wcukierski\\enron-email-dataset\\versions\\2\\emails.csv")

In [ ]:

print(df.head())  
print(df.info()) 
print(df.columns) 

In [ ]:
df['file'][0]

In [ ]:
from pprint import pprint
pprint(df['message'][30000])

In [ ]:
str(df['message'][30000]).split()

In [ ]:
# import enum
# import networkx as nx
# import matplotlib.pyplot as plt

# G = nx.DiGraph()
# df2 = df.iloc[:10000]
# senders = []
# recipients = []
# dict_sr = {}
# for i, (_, row) in enumerate(df2.iterrows()):
    
#     split_msg = str(row["message"]).split()
#     # print(i,row["message"])
#     # print(split_msg)
#     sender = split_msg[split_msg.index("From:")+1]
#     senders.append(sender)
#     if sender not in dict_sr:
#         dict_sr[sender] = []
#     #receiver = split_msg[split_msg.index("To:")+1]
#     # print(sender)
    
#     try:
#         for idx in range(split_msg.index("To:")+1, split_msg.index("Subject:")):
#             dict_sr[sender].append(split_msg[idx])
#     except:
#         for idx in range(split_msg.index("X-To:")+1, split_msg.index("Subject:")):
#             dict_sr[sender].append(split_msg[idx])
    
#     dict_sr[sender].sort()

#     # print(recipients)
#     # for recipient in recipients:
#     #     G.add_edge(sender, recipient)

# # plt.figure(figsize=(12, 8))
# # nx.draw(G, with_labels=True, node_size=1000, font_size=8, edge_color="gray")
# # plt.title("Enron Sender-Recipient Email Network")
# # plt.show()
# # print(senders,'\n',recipients)
# # print(len(senders),len(recipients),senders.count('phillip.allen@enron.com') / len(senders))


In [ ]:
import faiss
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# embedding_model = HuggingFaceEmbeddings()
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}  # set True to compute cosine similarity
model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
# Split email texts into chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=50)
email_texts = df["message"].iloc[:10000].dropna().tolist()
# email_chunks = text_splitter.split_text("\n".join(email_texts))

In [ ]:
len(email_texts)

In [ ]:
from langchain_core.documents import Document
import pprint
docslist = []
for i, ec in enumerate(email_texts):
    #pprint.pprint(ec.split())
    
    split_msg = ec.split()
    msg_start = split_msg.index("X-FileName:")
    sender = split_msg[split_msg.index("From:")+1]
    recips = []
    #print(f'EMAIL CHUNK {i}\n\n',ec.split()[msg_start+3:])
    try:
        for idx in range(split_msg.index("To:")+1, split_msg.index("Subject:")):
            recips.append(split_msg[idx])
    except:
        for idx in range(split_msg.index("X-To:")+1, split_msg.index("Subject:")):
            recips.append(split_msg[idx])

    date_idx = split_msg.index("Date:")+1
    date_idx_end = date_idx + 6
    full_content = " ".join(split_msg[msg_start:])
    #docslist.append(Document(page_content=full_content, metadata={"sender":sender,"recipient":" ".join(recips),"date":" ".join(split_msg[date_idx : date_idx_end])}))
    prefix = {"sender":sender,"recipient":" ".join(recips),"date":" ".join(split_msg[date_idx : date_idx_end])}
    #print(list(zip(prefix.keys(),prefix.values())))
    final_list = text_splitter.split_text(full_content)
    #print(list(zip(prefix.keys(),prefix.values()))+final_list)
    for f in final_list:
        docslist.append(str(list(zip(prefix.keys(),prefix.values())))+f)

In [ ]:
docslist

In [ ]:
len(docslist)

In [ ]:
final_doc_list = []

In [ ]:
for d in docslist:
    print(d)
    final_doc_list.append(Document(page_content=d))

In [ ]:
db = FAISS.from_documents(final_doc_list[:300], model)
db.save_local("email_faiss")

In [ ]:
docslist = final_doc_list[301:602]

In [ ]:
# batch_size = 20
# for i in range(0, len(docslist), batch_size):
#     batch_docs = docslist[i : i + batch_size]  
#     await db.aadd_documents(batch_docs)  
#     print(f"Batch {i // batch_size} added ({len(batch_docs)} docs).")

import asyncio
import concurrent.futures
from langchain_community.vectorstores import FAISS
def process_batch_sync(db, batch_docs, batch_id):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(db.aadd_documents(batch_docs))
    loop.close()
    print(f"✅ Batch {batch_id} added ({len(batch_docs)} docs).")

async def batch_insert(db, docslist, batch_size=20, num_workers=6):
    loop = asyncio.get_running_loop()
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        tasks = []
        
        for i in range(0, len(docslist), batch_size):
            batch_docs = docslist[i : i + batch_size]
            batch_id = i // batch_size
            tasks.append(loop.run_in_executor(executor, process_batch_sync, db, batch_docs, batch_id))
        
        await asyncio.gather(*tasks) 

# Run the optimized batch insertion
await batch_insert(db, docslist, batch_size=40)

In [ ]:
db.save_local("email_faiss")
print("FAISS index updated and saved.")

In [ ]:
db = FAISS.load_local("email_faiss",model,allow_dangerous_deserialization=True)

In [ ]:
from initialize_groq import init_groq
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
        """
            Answer question only provided the context. Give a detailed answer IN minimum 5 sentences!
            SAY I DONT KNOW IF CONTEXT IS NOT ENOUGH. DONT MAKE UP ANSWERS. BUT YOU ARE FREE TO INFER/SUGGEST.
            {context}

            Here is question:
            {input}
        """
)

retriever = db.as_retriever(search_kwargs={'k':20, 'search_type':'mmr','lambda_mult':0.2})

_, llm = init_groq(model_name="llama-3.3-70b-versatile")
import random
from initialize_groq import api_keys
llm.groq_api_key = random.choice(api_keys)
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Step 1: Retrieve Top-K Similar Documents (Initial Broad Search)
retriever_topk = db.as_retriever(search_kwargs={'k': 20, 'search_type': 'similarity'})  # Retrieve more docs first

# Step 2: Apply MMR for Diversity (Reduce Redundant Docs)
retriever_mmr = db.as_retriever(search_kwargs={'k': 20, 'search_type': 'mmr', 'lambda_mult': 0.2})  

# # Step 3: Apply Similarity Threshold to Remove Low-Relevance Docs
retriever_threshold = db.as_retriever(search_kwargs={'k': 20, 'search_type': 'similarity_score_threshold', 'score_threshold': -1})

# Create the Hybrid Retrieval Pipeline
retrieval_chain_topk = create_retrieval_chain(retriever_topk, document_chain)  # Initial broad search
retrieval_chain_mmr = create_retrieval_chain(retriever_mmr, document_chain)    # Apply MMR re-ranking
retrieval_chain_threshold = create_retrieval_chain(retriever_threshold, document_chain)  # Final filtering



In [ ]:
for d in docslist:
    print(d)

In [ ]:
import pprint
query = "give me 5 emails involving buck buckner of honeywell"
pprint.pprint(retrieval_chain_topk.invoke({"input":query}))

pprint.pprint(retrieval_chain_mmr.invoke({"input":query}))

# pprint.pprint(retrieval_chain_threshold.invoke({"input":query}))

In [ ]:
for doc in docslist:
    print(len(doc.page_content.split()))
    

In [ ]:
from langchain_core.tools import tool

# Ensure retrieval_chain is correctly defined before calling this tool
@tool
def ragtool(query: str) -> str:
    """
    This is a retrieval-augmented generation (RAG) tool that queries a vector store 
    containing Enron emails.
    
    Parameters:
    query (str): The input query for retrieval.
    
    Returns:
    str: The retrieved answer from the vector store.
    """
    try:
        answer = retrieval_chain_mmr.invoke({"input": query})['answer']
        return f"Here is the ANSWER. STOP AND WAIT FOR USER QUERY. \n```{answer}```"
    except Exception as e:
        return f"Error: Failed to retrieve answer. Details: {str(e)}"


In [ ]:
from typing import Literal
from langchain_core.runnables.history import RunnableWithMessageHistory
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain.memory import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langgraph.prebuilt import ToolNode, tools_condition



toolnode = ToolNode([ragtool])

def call_model(state: MessagesState):
    state["messages"]
    messages = state["messages"]
    #print(messages)
    llm.groq_api_key = random.choice(api_keys)
    llm_with_tool = llm.bind_tools([ragtool])
    response = llm_with_tool.invoke(messages)
    
    
    return {"messages": [response]}

from langgraph.graph import END
def router_function(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

memory = MemorySaver()
workflow = StateGraph(MessagesState)    
workflow.add_node("agent", call_model)
workflow.add_node(toolnode)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    router_function,
    {
       "tools": "tools",
       END: END,
    },
)
workflow.add_edge("tools", "agent")
app = workflow.compile(checkpointer=memory)



In [ ]:
from IPython.display import display_png
display_png(app.get_graph().draw_mermaid_png(),raw=True)

In [ ]:
import time
while True:
    theinput = input("Enter something: ")
    if 'exit' in theinput:
        break
    inp = {"messages":[theinput]}
    
    config = {"configurable": {"thread_id": 1}}
    events = app.stream(inp, config=config, stream_mode="values")

    for event in events:
        event["messages"][-1].pretty_print()
    time.sleep(1)

In [ ]:
# from typing import Literal, List
# from langchain_core.runnables.history import RunnableWithMessageHistory
# from langgraph.graph import StateGraph, START, END, MessagesState
# from langgraph.checkpoint.memory import MemorySaver
# from langchain.memory import ChatMessageHistory
# from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
# from langgraph.prebuilt import ToolNode
# from langchain_core.documents import Document
# from langchain_core.tools import tool
# from langchain.chat_models import ChatOpenAI
# from langchain_core.prompts import ChatPromptTemplate

# # === Define Custom Tools ===


# @tool
# def filter_emails_by_keyword(emails: List[Document], keywords: List[str]) -> List[Document]:
#     """Filter emails that contain the given keywords in the content or metadata."""
#     def filter_email(email):
#         content = email.page_content.lower()
#         metadata = " ".join(str(val).lower() for val in email.metadata.values())
#         return any(keyword.lower() in content or keyword.lower() in metadata for keyword in keywords)
    
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         filtered_emails = list(filter(None, executor.map(lambda e: e if filter_email(e) else None, emails)))

#     return filtered_emails

# @tool
# def filter_emails_by_metadata(
#     emails: List[Document], sender: str = None, recipient: str = None, date: str = None
# ) -> List[Document]:
#     """Filter emails by metadata fields like sender, recipient, or date."""
#     def filter_email(email):
#         if sender and email.metadata.get("sender", "").lower() != sender.lower():
#             return None
#         if recipient and recipient.lower() not in email.metadata.get("recipient", "").lower():
#             return None
#         if date and date not in email.metadata.get("date", ""):
#             return None
#         return email
    
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         filtered_emails = list(filter(None, executor.map(filter_email, emails)))
    
#     return filtered_emails

# @tool
# def summarize_emails(emails: List[Document]) -> List[Document]:
#     """Summarize emails before adding them to FAISS."""
#     _,llm = init_groq(model_name="llama-3.3-70b-versatile")
#     prompt = ChatPromptTemplate.from_template("Summarize the following email:\n{email}")
#     summarized_docs = []
#     for email in emails:
#         chain = prompt | llm
#         summary = chain.invoke(email.page_content)
#         summarized_docs.append(Document(page_content=summary.content, metadata=email.metadata))
#     return summarized_docs

# toolnode = ToolNode([ragtool, filter_emails_by_keyword, filter_emails_by_metadata, summarize_emails])
# llm_with_tool = llm.bind_tools([ragtool, filter_emails_by_keyword, filter_emails_by_metadata, summarize_emails])

# # === Define Model Function ===
# def call_model(state: MessagesState):
#     """Modify agent behavior to apply filtering and summarization before RAG."""
#     messages = state["messages"]
#     query = messages[-1]

#     # Step 1: Apply Keyword Filtering
#     filtered_emails = filter_emails_by_keyword.invoke({"emails": docslist, "keywords": [query.content]})

#     # Step 2: Apply Metadata Filtering
#     filtered_emails = filter_emails_by_metadata.invoke(
#         {"emails": filtered_emails, "sender": "", "recipient": "", "date": ""}
#     )

#     # Step 3: Summarize Emails if Needed
#     summarized_emails = summarize_emails.invoke({"emails": filtered_emails})

#     # Step 4: Run RAG Tool on Filtered Emails
#     state["messages"].append("\n")
#     response = llm_with_tool.invoke([summarized_emails])

#     return {"messages": [response]}

# # === Define Router Function ===
# def router_function(state: MessagesState) -> Literal["tools", END]:
#     messages = state["messages"]
#     last_message = messages[-1]
#     if last_message.tool_calls:
#         return "tools"
#     return END

# # === Build LangGraph Workflow ===
# memory = MemorySaver()
# workflow = StateGraph(MessagesState)
# workflow.add_node("agent", call_model)
# workflow.add_node(toolnode)
# workflow.add_edge(START, "agent")
# workflow.add_conditional_edges(
#     "agent",
#     router_function,
#     {
#         "tools": "tools",
#         END: END,
#     },
# )
# workflow.add_edge("tools", "agent")
# app = workflow.compile(checkpointer=memory)

# while True:
#     theinput = input("Enter something: ")
#     if 'exit' in theinput:
#         break
#     inp = {"messages":[theinput]}

#     config = {"configurable": {"thread_id": 1}}
#     events = app.stream(inp, config=config, stream_mode="values")

#     for event in events:
#         event["messages"][-1].pretty_print()


In [ ]:
while True:
    theinput = input("Enter something: ")
    if 'exit' in theinput:
        break
    inp = {"messages":[theinput]}

    config = {"configurable": {"thread_id": 1}}
    events = app.stream(inp, config=config, stream_mode="values")

    for event in events:
        event["messages"][-1].pretty_print()